### Prospeção de Dados 2023/2024 - Course Project
#### Professor: André Falcão
#### Alunos: 

Catherine Prokhorov (62608) - XX Hours <br>
Guilherme Cepeda (62931) - XX Hours <br>
Jorge Aleluia (54549) - XX Hours <br>
Rómulo Nogueira (56935) - XX Hours <br>

#### Library

In [1]:
import pickle
import pandas as pd

In [8]:
## import the data

train_columns = ['mol_id', 'chembl_id', 'activity']
test_columns = ['mol_id', 'chembl_id', 'activity']


activity_train = pd.read_csv("activity_train.csv", names = train_columns, header = None)
activity_test_blanked = pd.read_csv("activity_test_blanked.csv", names = test_columns, header = None)

with open("mol_bits.pkl", "rb") as f:
    mol_bits = pickle.load(f)

In [15]:
print(activity_train.head())
print(activity_test_blanked.head())

   mol_id       chembl_id  activity
0  O14842   CHEMBL2022243         4
1  O14842   CHEMBL2022244         6
2  O14842   CHEMBL2022245         2
3  O14842   CHEMBL2022246         1
4  O14842   CHEMBL2022247         4
   mol_id       chembl_id  activity
0  O14842   CHEMBL2022258         0
1  O14842   CHEMBL2047161         0
2  O14842   CHEMBL2047163         0
3  O14842   CHEMBL2047168         0
4  O14842   CHEMBL2047169         0


In [14]:
X_train = activity_train.drop(columns=['activity'])
y_train = activity_train['activity']

# print(X_train.head())
# print(y_train.head())

   mol_id       chembl_id
0  O14842   CHEMBL2022243
1  O14842   CHEMBL2022244
2  O14842   CHEMBL2022245
3  O14842   CHEMBL2022246
4  O14842   CHEMBL2022247
0    4
1    6
2    2
3    1
4    4
Name: activity, dtype: int64
